<img align="right" src="https://www.universidadviu.com/sites/universidadviu.com/themes/custom/sbx_theme/logo.webp" width="175">

<div style="clear: both; margin: 0; padding: 0; line-height: 0;"></div>

<hr style="border: 0; height: 4px; background: #E65015; margin-top: 0px;">

# <span style="font-size: 3rem; font-weight: bold;">Algoritmos de Optimización - Seminario</span>

## <span style="font-size: 1.5rem;">Oleksandr Yatsentyuk y David Bouzas i Morales</span>  
## <span style="font-size: 1.5rem;">Domingo, 8 de febrero de 2026</span>

---
### **URL**
- https://github.com/
- https://github.com/

### **Problema**
Sesiones de doblaje

### **Descripción del Problema**
Se precisa coordinar el doblaje de una película. Los actores del doblaje deben coincidir en las tomas en las que sus personajes aparecen juntos en las diferentes tomas. Los actores de doblaje cobran todos la misma cantidad por cada día que deben desplazarse hasta el estudio de grabación independientemente del número de tomas que se graben. No es posible grabar más de 6 tomas por día. El objetivo es planificar las sesiones por día de manera que el gasto por los
servicios de los actores de doblaje sea el menor posible. Los datos son:

- Número de actores: 10
- Número de tomas: 30
- Actores/Tomas: https://bit.ly/36D8IuK
  - 1 indica que el actor participa en la toma
  - 0 en caso contrario


# Índice
0. [Pasos previos](#previos)
1. [Posibilidades con y sin restricciones](#posibilidades)
2. [Estructura de Datos](#estructura)
3. [Función objetivo](#objetivo)
4. [Fuerza bruta](#fuerzaBruta)
5. [Algoritmo que mejora la complejidad](#mejora)
6. [Juego de datos de entrada aleatorios](#juegoAleatorio)
7. [Referencias](#referencias)
8. [Estudio del Problema](#estudio)

## 0. Pasos previos <a id="previos"></a>

In [ ]:
# Librerías necesarias para el proyecto
import math
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
# URL con los datos del problema de doblaje
url = "https://docs.google.com/spreadsheets/d/1Ipn6IrbQP4ax8zOnivdBIw2lN0JISkJG4fXndYd27U0/export?format=csv"

# Leer el CSV
df = pd.read_csv(
    url,                           # URL donde obtener el dataset
    header = 1,                    # La 1ra fila solo hay el título "Actor", empezar por la fila 2
    index_col = 0,                 # La 1ra columna como índice del DataFrame (Toma)
    usecols = list(range(0, 11)),  # Leer de las columnas 0 a 10
    nrows = 30                     # Leer las primeras 30 filas
).add_prefix("Actor ")             # Añade el prefijo 'Actor ' a todas las columnas

In [ ]:
# Asignar el nombre 'Toma' al índice del DataFrame
df.index.name = 'Toma'

# Nueva columna con el total de actores que participan en cada toma
df['Total'] = df.sum(axis = 1)

# Mostrar el DataFrame resultante
display(df)

,Actor 1,Actor 2,Actor 3,Actor 4,Actor 5,Actor 6,Actor 7,Actor 8,Actor 9,Actor 10,Total
Toma,,,,,,,,,,,
1,1,1,1,1,1,0,0,0,0,0,5
2,0,0,1,1,1,0,0,0,0,0,3
3,0,1,0,0,1,0,1,0,0,0,3
4,1,1,0,0,0,0,1,1,0,0,4
5,0,1,0,1,0,0,0,1,0,0,3
6,1,1,0,1,1,0,0,0,0,0,4
7,1,1,0,1,1,0,0,0,0,0,4
8,1,1,0,0,0,1,0,0,0,0,3
9,1,1,0,1,0,0,0,0,0,0,3


## 1. Posibilidades con y sin restricciones <a id="posibilidades"></a>

- **(*) ¿Cuántas posibilidades hay sin tener en cuenta las restricciones?**

Sea $n \in \mathbb{N}$ el número de tomas a grabar, para calcular el número de posibilidades sin tener en cuenta las restricciones es necesario estudiar el orden de las tomas a grabar y la distribución de las sesiones de grabación a lo largo de los días:

**Orden de las tomas**

- Si asumimos que el total de ordenaciones posibles es relevante, entonces se trata de un problema de **Permutación sin Repetición** y, por lo tanto, el número de posibilidades que tendremos es:
$$
P(n) = n!
$$

**Distribución de las sesiones de grabación**

- Para cada sesión de grabación de un día se pueden grabar un **número k distinto** de tomas: $k=\{1,2, \ldots, n\}$.

- Las grabaciones son **únicas**, es decir, no pueden repetirse. Por lo tanto, una vez grabada una toma deberá ser excluida del conjunto de tomas a grabar en las próximas sesiones.

- Sea $d\in \{0, \ldots, n-1\}$ el **número de particiones** en que se pueden distribuir las sesiones de grabación de las tomas. Por ejemplo, si $d=0$ significa que todas las tomas se han grabado en un mismo día (no ha ninguna partición). En cambio, si $d=1$ significa que las tomas se han grabado en dos sesiones distintas, y así sucesivamente.

- Es importante mencionar que el número de posibles ordenación de la partición $d=0$ es $n-1 \choose 0$, el de la partición $d=1$ es $n-1 \choose 1$, y así hasta el último número combinatorios. Por tanto, el número total de posibles particiones es la suma de todas las posibles distribuciones de las particiones, es decir, $\sum_{j=0}^{n-1}{{n-1 \choose m}}$.

**Resultado**

El número total de posibles ordenaciones de las tomas a lo largo de los días se obtendrá calculando el producto del orden de las tomas por la distribución de las sesiones de grabación, que por identidad bionomial es

$$ \text{Núm. posibilidades} = {n! \cdot \sum_{j=0}^{n-1}{{n-1 \choose j}}} = {n! \cdot 2^{n-1}}$$

Así, para $n = 30$ tomas el número de posibilidades sin restricciones es:

In [ ]:
n = 30
print('Para n=30, el número de posibilidades sin restricciones es', math.factorial(n) * (2**(n-1)))

Para n=30, el número de posibilidades sin restricciones es 142406544757979162368320409970933760000000


- **¿Cuantas posibilidades hay teniendo en cuenta todas las restricciones?**

Sean $n, k \in \mathbb{N}, 1\leq k \leq n$ el número total de tomas a grabar y el número de tomas a grabar por día, respectivamente, para calcular el número de posibilidades es necesario estudiar el orden de las tomas a grabar y la distribución de las sesiones de grabación a lo largo de los días.

**Orden de las tomas**

- Si asumimos que el número total de ordenaciones posibles es relevante, implica que al igual que antes, se trata de un problema de permutaciones sin repetición y el numero de posibilidades es tendremos $n!$.

- Dado que no hay restricciones sobre como deben ser grabadas las tomas de la sesión de un día, no sabemos si el orden de las $k$ tomas importa. Supongamos, entonces, que el orden de grabación de las tomas es indiferente.

**Distribución de las sesiones de grabación**

- En cada sesión de grabación de un día se pueden grabar un número distinto de tomas $k=\{1,2, \ldots, n\}$.

- El mínimo número de sesiones en que podemos grabar todas las tomas es de $r = \frac{n}{k}$ días. No obstante, podemos formar combinaciones de sesiones con diferentes números de tomas, es decir, $\leq k$.

- Las grabaciones son únicas y, por lo tanto, no se pueden repetir. Lo que implica que una vez grabada una toma deberá ser excluída del conjunto de tomas a grabar en las siguientes sesiones.

- Si fijamos el número tomas rodadas por sesión para que sea el mismo en todas las sesiones, entonces tendremos que ${n-m \cdot j \choose k}$ posibilidades, done $j = \{1,2,\ldots,k\}$ es el $j$-ésimo día de grabación.


**Resultado**

El número total de posibles ordenaciones de las tomas a lo largo de los días para un número de tomas $k$ fijo se obtendrá calculando
El número total de posibles ordenaciones de las tomas a lo largo de los días para un número de tomas $k$ fijo se obtendrá calculando

$$ \text{Núm. posibilidades} = \prod_{j=1}^{\frac{n}{k}}{{n-k \cdot j \choose k}} $$

Así, si $n = 30$ tomas y $k = 6$ el número total de tomas que se puede grabar en un día, entonces el número de soluciones posibles con restriccions es

In [ ]:
n = 30
k = 6
num_posibilidades = 1
for j in range(0, k-1):
    num_posibilidades *= math.comb(n-k*j, k)

print('Para n=30 y k=6, el número de posibilidades con restricciones es', num_posibilidades)

Para n=30 y k=6, el número de posibilidades con restricciones es 1370874167589326400


## 2. Estructura de Datos <a id="estructura"></a>

- **(*) ¿Cual es la estructura de datos que mejor se adapta al problema? Argumentalo.(Es posible que hayas elegido una al principio y veas la necesidad de cambiar, arguentalo)**

**Primera Aproximación**

En una primera aproximación, la idea inicial era representar los **datos de entrada** del problema de planificación de tomas y actores mediante una estructura matricial clara y formal, utilizando herramientas habituales en análisis de datos como un **DataFrame** de pandas o un **array** de NumPy. Esta decisión responde a la necesidad de trabajar con una representación estructurada, eficiente y fácilmente manipulable mediante operaciones matriciales.

Cada celda de la matriz contendría un valor indicador definido de la siguiente manera:
- un 1 si el actor j participa en la toma i
- un 0 en caso contrario.

Esta representación equivale a una **matriz de incidencia entre tomas y actores**, lo que nos permite visualizar de manera inmediata qué actores intervienen en cada toma y, de forma inversa, en qué tomas participa cada actor.

Para los **datos de salida** nuestra idea era presentar la solución se como una lista de sublistas donde cada sublista estaría formada por un número de tomas, más pequeño o igual a 6, para una sesión específica y se presentarían tantas sublistas como sesiones de grabación se considerase.

**Aproximación Final**

In [ ]:
# Método que transforma la matriz de datos original en un formato compacto de máscaras de bits para optimizar el rendimiento
def build_take_masks(df):
    take_masks = {}

    for take, row in df.iterrows():
        mask = 0

        for i, col in enumerate(df.columns):
            if int(row[col]) == 1:
                mask |= (1 << i)
        take_masks[int(take)] = mask

    return take_masks

## 3. Función objetivo <a id="objetivo"></a>

- **(*) ¿Cual es la función objetivo?**

La función objetivo busca minimizar el sumatorio de los actores presentes en cada día de grabación. Dado que todos cobran lo mismo, el "salario" puede simplificarse a una unidad ($1$) si solo queremos minimizar el número de "presencias". Otra opción sería mantenerlo como una constante ($S$). Si definimos:

- $i$: Índice del actor ($1$ a $10$).
- $j$: Índice de la toma ($1$ a $30$).
- $k$: Índice del día/sesión de grabación.
- $A_{i,j}$: Matriz binaria (1 si el actor $i$ participa en la toma $j$).
- $X_{j,k}$: Variable de decisión (1 si la toma $j$ se graba el día $k$).

La función a minimizar es:$$\text{Minimizar } Z = \sum_{k} \sum_{i} \text{Participa}(i, k)$$Donde $\text{Participa}(i, k)$ es una variable binaria que vale 1 si el actor $i$ interviene en al menos una de las tomas programadas para el día $k$.

La función objetivo mide el coste total de contratación derivado de una planificación específica de las tomas. Este coste se calcula como la suma de los costes individuales de cada sesión diaria. Para evaluarlo, hemos implementado el método **calculo_coste()** que procesa una propuesta de calendario (tomas asignadas a cada día) y determina el gasto acumulado basándose en la presencia diaria de los actores en el estudio.

El método **calculo_coste()** recibe como entrada una permutación de las tomas y la matriz de participación de los actores. Su funcionamiento se basa en una segmentación por bloques: Inicialmente, agrupa las tomas en sesiones de un máximo de 6 unidades siguiendo el orden del vector solución. Para cada sesión, el algoritmo realiza una operación lógica de "OR" sobre las columnas de los actores. Esto nos permite identificar de forma binaria si un actor debe desplazarse al estudio ese día, independientemente de si participa en una o en seis tomas. Finalmente, contabiliza los actores presentes por día, los multiplica por su salario y acumula estos valores para devolver el coste total de la solución, permitiendonos así comparar diferentes planificaciones bajo un mismo criterio económico:

In [ ]:
# Método que calcula el costo de la solución
def calculo_costo_(vector_solucion, datos_tomas, salario=1):
    costo_total = 0
    # Bloques de 6 tomas por día
    for i in range(0, len(vector_solucion), 6):
        sesion_indices = vector_solucion[i:i+6]
        tomas_del_dia = datos_tomas[sesion_indices]

        # Un actor cobra si participa en como mínimo una toma (suma por columnas > 0)
        actores_presentes = np.any(tomas_del_dia == 1, axis=0)
        costo_dia = np.sum(actores_presentes) * salario
        costo_total += costo_dia

    return costo_total

Un detalle importante de mencionar del código es que el uso del parámetro **axis=0** en la operación lógica permite evaluar la presencia de los actores por columnas. Esto garantiza que, si un actor participa en varias tomas dentro de una misma sesión, el sistema lo contabilice como una única asistencia diaria, quedando reflejado que el salario se devenga por día de desplazamiento y no por el volumen de tomas grabadas.

- **(*) ¿Es un problema de maximización o minimización?**

Como hemos explicado en el anterior apartado se trata de un problema de **minimización**, ya que el objetivo principal es reducir al máximo el gasto total por los servicios de los actores. Dado que cada actor cobra una cantidad fija por cada día que debe desplazarse al estudio (independientemente de cuántas tomas grabe ese día), la estrategia de optimización consiste en agrupar las tomas de manera que el número total de asistencias diarias de los actores sea el **menor posible**.

En términos matemáticos, buscamos el valor más bajo de la función de coste:

$$\text{Minimizar } Z = \sum_{k=1}^{n} \text{Coste}_k$$

Donde $n$ es el número total de días de grabación y $\text{Coste}_k$ es el coste de la sesión $k$ basado en los actores que intervienen ese día.

## 4. Fuerza bruta <a id="fuerzaBruta"></a>

- **Diseña un algoritmo para resolver el problema por fuerza bruta.**

In [ ]:
def brute_force_best_schedule_from_masks(take_masks, max_takes_per_day=6):
    takes = sorted(take_masks.keys())
    best_cost = float("inf")
    best_schedule = None
    days = []

    def rec(idx, current_cost):
        nonlocal best_cost, best_schedule
        if current_cost >= best_cost: return # Poda
        if idx == len(takes):
            best_cost = current_cost
            best_schedule = [list(d["takes"]) for d in days]
            return

        take = takes[idx]
        tmask = take_masks[take]

        # Intentar en días existentes
        for d in days:
            if len(d["takes"]) < max_takes_per_day:
                old_mask = d["mask"]
                new_mask = old_mask | tmask
                delta = new_mask.bit_count() - old_mask.bit_count()

                d["takes"].append(take)
                d["mask"] = new_mask
                rec(idx + 1, current_cost + delta)
                d["takes"].pop()
                d["mask"] = old_mask

        # Intentar crear un día nuevo
        days.append({"takes": [take], "mask": tmask})
        rec(idx + 1, current_cost + tmask.bit_count())
        days.pop()

    rec(0, 0)
    return best_schedule, best_cost

- **Calcula la complejidad del algoritmo por fuerza bruta.**

El algoritmo asigna las $n=30$ tomas una a una a “días” (grupos) con capacidad $\le 6$. Como solo permite crear un nuevo día al final (append), evita duplicados por permutación de días y, por tanto, explora **particiones no etiquetadas** (no el caso etiquetado). El tamaño del espacio factible en el peor caso viene dado por el número de particiones restringidas:
$$F(30)=\sum_{k=5}^{30} T(30,k)\approx 7.264\times 10^{23},$$
donde $T(n,k)$ cuenta particiones de $n$ tomas en $k$ días con tamaños $\le 6$ (no es el Stirling estándar $S(n,k)$, porque aquí hay cota de tamaño por bloque).

El árbol de búsqueda tiene profundidad $n$ y en un estado parcial con $k$ días abiertos, la siguiente toma tiene hasta $k$ intentos de inserción (si hay capacidad) más 1 intento de crear día nuevo, por lo que el coste local por nodo está dominado por el bucle sobre días. Con máscaras de 10 bits, las operaciones OR y `bit_count()` son de coste constante, así que el coste local por expansión es
$$C_{\text{nodo}}=O(k)\le O(n).$$
Una cota superior razonable para el número total de nodos visitados sin poda efectiva es proporcional al número de soluciones completas por el número de niveles:
$$\#\text{nodos}\le O(n\cdot F(30)).$$
Por tanto, la complejidad temporal en peor caso queda acotada por:
$$T(n)=O\big(n\cdot F(n)\cdot n\big)=O\big(n^2,F(n)\big),$$
y para $n=30$:
$$T(30)=O\big(30^2\cdot 7.264\times 10^{23}\big)\approx O(6.5\times 10^{26})\ \text{operaciones básicas}.$$
La poda branch-and-bound ($\text{si } current_cost\ge best_cost$) puede reducir mucho el tiempo en práctica si se encuentra pronto una buena cota, pero **no cambia el peor caso**.

La complejidad espacial es lineal: la recursión tiene profundidad $n$ y la estructura `days` almacena en total $n$ asignaciones y hasta $n$ máscaras:
$$S(n)=O(n).$$

## 5. Algoritmo que mejora la complejidad <a id="mejora"></a>

- **(*) Diseña un algoritmo que mejore la complejidad del algortimo por fuerza bruta. Argumenta porque crees que mejora el algoritmo por fuerza bruta.**

In [ ]:
# Respuesta

- **(*) Calcula la complejidad del algoritmo.**

In [ ]:
# Respuesta

## 6. Juego de datos de entrada aleatorios <a id="juegoAleatorio"></a>

- **Según el problema (y tenga sentido), diseña un juego de datos de entrada aleatorios.**

A continuación, vamos a implementar una función diseñada para generar escenarios de prueba de forma dinámica y escalable, permitiendo evaluar el rendimiento de los algoritmos bajo diferentes volúmenes de datos. Este método utiliza la librería NumPy para crear una matriz de participación aleatoria basada en una densidad específica y, posteriormente, aplica la lógica de empaquetado de bits para consolidar la información de cada toma en un único valor entero. Gracias a este enfoque, podemos simular rápidamente rodajes de cualquier magnitud manteniendo la eficiencia computacional que caracteriza a nuestra estructura de máscaras de bits:

In [ ]:
# Genera un diccionario de máscaras de bits optimizado para cualquier volumen
# La densidad indica el porcentaje de que un actor aparezca en el total de tomas (30%)
def generar_datos_aleatorios(n_tomas=30, n_actores=10, densidad=0.3):
    matriz = (np.random.rand(n_tomas, n_actores) < densidad).astype(int)

    take_masks = {}
    for i, fila in enumerate(matriz):
        mask = 0
        for bit_idx, valor in enumerate(fila):
            if valor == 1:
                mask |= (1 << bit_idx) # Desplazamiento de bits

        # Guardamos con ID de toma 1..N
        take_masks[i + 1] = mask

    return take_masks, n_actores

- **Aplica el algoritmo al juego de datos generado.**

Para complementar las herramientas de análisis, implementamos un método de resolución exacta mediante fuerza bruta optimizada con el fin de obtener el coste mínimo absoluto en escenarios de menor escala. Esta función integra el generador de datos aleatorios con el algoritmo de búsqueda recursiva, permitiendo validar la precisión de nuestras aproximaciones heurísticas al comparar sus resultados con el óptimo global:

In [ ]:
def solve_random_brute_force(n_tomas=20, n_actores=10, max_takes_per_day=6):
    take_masks, _ = generar_datos_aleatorios(n_tomas, n_actores)

    mejor_plan, coste_optimo = brute_force_best_schedule_from_masks(
        take_masks,
        max_takes_per_day
    )

    return mejor_plan, coste_optimo

# Ejemplo de ejecución para el caso del problema (20 tomas, 10 actores) porque con fuerza bruta tardaría demasiado
resultado_optimo, coste_real = solve_random_brute_force(20, 10)

print(f"Optimización finalizada. Coste total: {coste_real}")
for i, sesion in enumerate(resultado_optimo):
    print(f"Sesión {i+1}: Tomas {sesion}")

Optimización finalizada. Coste total: 24
Sesión 1: Tomas [1, 3, 7, 11]
Sesión 2: Tomas [2, 4, 5, 9, 10, 20]
Sesión 3: Tomas [6, 8, 14, 16]
Sesión 4: Tomas [12, 13, 15, 17, 18, 19]


Finalmente, implementamos el método que genera un escenario aleatorio y aplica primero una estrategia codiciosa para establecer una base sólida, la cual es posteriormente optimizada mediante una búsqueda local intensiva que explora miles de combinaciones en milisegundos:

In [ ]:
def solve_random_scenario(n_tomas=30, n_actores=10, max_iter=10000):
    take_masks, _ = generar_datos_aleatorios(n_tomas, n_actores)

    days0 = greedy_schedule(take_masks, max_takes_per_day=6)

    mejores_dias, coste_final = local_search(
        days0,
        take_masks,
        max_takes_per_day=6,
        max_iter=max_iter
    )

    return mejores_dias, coste_final

# Ejemplo de ejecución para el caso del problema (30 tomas, 10 actores)
resultado, coste = solve_random_scenario(30, 10)

print(f"Optimización finalizada. Coste total: {coste}")
for i, dia in enumerate(resultado):
    print(f"Día {i+1}: Tomas {dia['takes']} | Coste día: {dia['cost']}")

## 7. Referencias <a id="referencias"></a>

## 8. Estudio del Problema <a id="estudio"></a>

- **Describe brevemente las lineas de como crees que es posible avanzar en el estudio del problema. Ten en cuenta incluso posibles variaciones del problema y/o variaciones al alza del tamaño.**

El siguiente paso para poder seguir avanzando en el estudio de este problema es la transición hacia metaheurísticas de trayectoria como, por ejemplo, el **Simulated Annealing (Enfriamiento Simulado)** o el **Tabu Search**. Estas técnicas permitirían que el algoritmo actual de búsqueda local salte de los óptimos locales, aceptando temporalmente soluciones ligeramente peores para explorar zonas más profundas del espacio de búsqueda. Esto es vital cuando el número de tomas aumenta significativamente, ya que los algoritmos codiciosos suelen quedar atrapados en configuraciones que parecen buenas pero no son las mejores globalmente.

Finalmente, ante un aumento masivo del tamaño del problema, se podría explorar la computación paralela y el uso de redes neuronales por refuerzo. Dado que actualmente usamos una estructura de máscaras de bits, estas operaciones podrían distribuirse en miles de núcleos de una GPU para evaluar millones de combinaciones por segundo.